# DCGAN trên CIFAR10

Notebook này huấn luyện DCGAN trên CIFAR10 (ảnh màu 32×32, resize lên 64×64).

In [1]:
# !pip install torch torchvision matplotlib

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
# Cấu hình
ROOT_PATH = "/kaggle/working/"
EPOCHS = 50
BATCH_SIZE = 128
NZ = 100
LR = 2e-4
OUTPUT_DIR = ROOT_PATH + "outputs_dcgan_cifar10"


In [4]:
class DCGAN_G(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.main(z)


class DCGAN_D(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.main(x)
        return out.view(-1, 1)


In [5]:
def sample_noise(batch_size, nz, device):
    return torch.randn(batch_size, nz, 1, 1, device=device)


def train_dcgan_cifar10(epochs=50, batch_size=128, nz=100, lr=2e-4, outdir="outputs_dcgan_cifar10"):
    os.makedirs(outdir, exist_ok=True)

    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ])

    dataset = datasets.CIFAR10(root="data", train=True, transform=transform, download=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    G = DCGAN_G(nz=nz, ngf=64, nc=3).to(device)
    D = DCGAN_D(nc=3, ndf=64).to(device)

    criterion = nn.BCELoss()
    optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

    fixed_noise = sample_noise(64, nz, device)

    for epoch in range(epochs):
        for i, (real_imgs, _) in enumerate(loader):
            real_imgs = real_imgs.to(device)
            cur_bs = real_imgs.size(0)

            optimizer_D.zero_grad()
            labels_real = torch.ones(cur_bs, 1, device=device)
            labels_fake = torch.zeros(cur_bs, 1, device=device)

            out_real = D(real_imgs)
            loss_real = criterion(out_real, labels_real)

            z = sample_noise(cur_bs, nz, device)
            fake_imgs = G(z).detach()
            out_fake = D(fake_imgs)
            loss_fake = criterion(out_fake, labels_fake)

            loss_D = loss_real + loss_fake
            loss_D.backward()
            optimizer_D.step()

            optimizer_G.zero_grad()
            z = sample_noise(cur_bs, nz, device)
            fake_imgs = G(z)
            out_fake_for_G = D(fake_imgs)
            loss_G = criterion(out_fake_for_G, labels_real)
            loss_G.backward()
            optimizer_G.step()

            if i % 200 == 0:
                print(f"[DCGAN CIFAR10] Epoch [{epoch+1}/{epochs}] "
                      f"Step [{i}/{len(loader)}] "
                      f"Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

        with torch.no_grad():
            fake = G(fixed_noise).cpu()
            fake = (fake + 1) / 2
            grid = make_grid(fake, nrow=8)
            save_image(grid, os.path.join(outdir, f"dcgan_cifar10_epoch_{epoch+1:03d}.png"))

    print("Hoàn thành huấn luyện DCGAN, ảnh lưu trong:", outdir)


In [6]:
train_dcgan_cifar10(epochs=EPOCHS, batch_size=BATCH_SIZE, nz=NZ, lr=LR, outdir=OUTPUT_DIR)

100%|██████████| 170M/170M [00:03<00:00, 47.3MB/s] 


[DCGAN CIFAR10] Epoch [1/50] Step [0/391] Loss_D: 1.3010, Loss_G: 2.3845
[DCGAN CIFAR10] Epoch [1/50] Step [200/391] Loss_D: 0.4137, Loss_G: 2.9980
[DCGAN CIFAR10] Epoch [2/50] Step [0/391] Loss_D: 0.6850, Loss_G: 2.8974
[DCGAN CIFAR10] Epoch [2/50] Step [200/391] Loss_D: 0.2960, Loss_G: 3.2473
[DCGAN CIFAR10] Epoch [3/50] Step [0/391] Loss_D: 0.5362, Loss_G: 3.2753
[DCGAN CIFAR10] Epoch [3/50] Step [200/391] Loss_D: 0.2948, Loss_G: 2.8074
[DCGAN CIFAR10] Epoch [4/50] Step [0/391] Loss_D: 0.3204, Loss_G: 2.6892
[DCGAN CIFAR10] Epoch [4/50] Step [200/391] Loss_D: 0.6473, Loss_G: 2.7644
[DCGAN CIFAR10] Epoch [5/50] Step [0/391] Loss_D: 1.1470, Loss_G: 5.9243
[DCGAN CIFAR10] Epoch [5/50] Step [200/391] Loss_D: 0.2316, Loss_G: 3.8431
[DCGAN CIFAR10] Epoch [6/50] Step [0/391] Loss_D: 0.6597, Loss_G: 4.6376
[DCGAN CIFAR10] Epoch [6/50] Step [200/391] Loss_D: 0.0860, Loss_G: 4.3212
[DCGAN CIFAR10] Epoch [7/50] Step [0/391] Loss_D: 0.0071, Loss_G: 8.9311
[DCGAN CIFAR10] Epoch [7/50] Step [200/